In [18]:
import pandas as pd

In [19]:
file1 = "crimedatadate.csv"
file2 = "weatherdata.csv"

# Import the data into a Pandas DataFrame
df = pd.read_csv(file1)
weather = pd.read_csv(file2)
df.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location,Day
0,0,10000092,HY189866,2015-03-18 19:44:00,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,...,25.0,04B,1144606.0,1903566.0,2015,02/10/2018 03:50:01 PM,41.891399,-87.744385,"(41.891398861, -87.744384567)",2015-03-18
1,1,10000094,HY190059,2015-03-18 23:00:00,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,...,67.0,26,1166468.0,1860715.0,2015,02/10/2018 03:50:01 PM,41.773372,-87.665319,"(41.773371528, -87.665319468)",2015-03-18
2,2,10000095,HY190052,2015-03-18 22:45:00,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,...,39.0,08B,1185075.0,1875622.0,2015,02/10/2018 03:50:01 PM,41.813861,-87.596643,"(41.81386068, -87.596642837)",2015-03-18
3,3,10000096,HY190054,2015-03-18 22:30:00,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,...,40.0,08B,1178033.0,1870804.0,2015,02/10/2018 03:50:01 PM,41.800802,-87.622619,"(41.800802415, -87.622619343)",2015-03-18
4,4,10000097,HY189976,2015-03-18 21:00:00,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,...,25.0,03,1144920.0,1898709.0,2015,02/10/2018 03:50:01 PM,41.878065,-87.743354,"(41.878064761, -87.743354013)",2015-03-18


In [20]:
del weather["Unnamed: 0"]
weather = weather.rename(columns={'Dates':'Day'})

In [21]:
df2 = pd.merge(df, weather[["Day", 'MinTemp', 'MaxTemp']], on="Day")
df2["Date"] = pd.to_datetime(df2["Date"])
df2['Month'] = df2['Date'].apply(lambda dt: (dt.month%12 + 3)//3)

seasons = [0,1,2,3,4]
season_names = ['Winter', 'Spring', 'Summer', 'Fall']

df2['Season'] = pd.cut(df2['Month'], seasons, labels=season_names)
df2.drop('Month', 1, inplace=True)

temps = df2.loc[: , "MinTemp":"MaxTemp"]
df2['AvgTemp'] = temps.mean(axis=1)
temp = [-200,30,50,70,90,200]
temp_labels = ['Below freezing', '30-50 degrees', '50-70 degrees', '70-90 degrees', 'Over 90 degrees']
df2['Weather'] = pd.cut(df2['AvgTemp'], temp, labels=temp_labels)

location_dict = {'RESIDENCE' : 'Inside',
                 'APARTMENT' : 'Inside',
                 'SIDEWALK' : 'Outside',
                 'STREET' : 'Outside',
                 'ALLEY' : 'Outside',
                 'SCHOOL, PUBLIC, BUILDING' : 'Inside',
                 'VEHICLE NON-COMMERCIAL' : 'Outisde',
                 'RESTAURANT' : 'Inside',
                 'GAS STATION' : 'Outside',
                 'RESIDENTIAL YARD (FRONT/BACK)' : 'Outside',
                 'PARK PROPERTY' : 'Outside',
                 'COMMERCIAL / BUSINESS OFFICE' : 'Inside',
                 'CTA PLATFORM' : 'Outside',
                 'SCHOOL, PUBLIC, GROUNDS' : 'Outside',
                 'VACANT LOT/LAND' : 'Outside',
                 'DEPARTMENT STORE' : 'Inside',
                 'HOTEL/MOTEL' : 'Inside',
                 'CHA HALLWAY/STAIRWELL/ELEVATOR' : 'Inside',
                 'BANK' : 'Inside' }
df2["Inside/Outside"] = df2["Location Description"].map(location_dict)
df2.head()

,Unnamed: 0,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,...,Latitude,Longitude,Location,Day,MinTemp,MaxTemp,Season,AvgTemp,Weather,Inside/Outside
0,0,10000092,HY189866,2015-03-18 19:44:00,047XX W OHIO ST,041A,BATTERY,AGGRAVATED: HANDGUN,STREET,False,...,41.891399,-87.744385,"(41.891398861, -87.744384567)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Outside
1,1,10000094,HY190059,2015-03-18 23:00:00,066XX S MARSHFIELD AVE,4625,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,...,41.773372,-87.665319,"(41.773371528, -87.665319468)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Outside
2,2,10000095,HY190052,2015-03-18 22:45:00,044XX S LAKE PARK AVE,0486,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,...,41.813861,-87.596643,"(41.81386068, -87.596642837)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Inside
3,3,10000096,HY190054,2015-03-18 22:30:00,051XX S MICHIGAN AVE,0460,BATTERY,SIMPLE,APARTMENT,False,...,41.800802,-87.622619,"(41.800802415, -87.622619343)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Inside
4,4,10000097,HY189976,2015-03-18 21:00:00,047XX W ADAMS ST,031A,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,...,41.878065,-87.743354,"(41.878064761, -87.743354013)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Outside


In [22]:
del df2['Unnamed: 0']
del df2['X Coordinate']
del df2['Y Coordinate']
del df2['Latitude']
del df2['Longitude']
del df2['FBI Code']
del df2['Case Number']
del df2['IUCR']


In [23]:
df2.head()

,ID,Date,Block,Primary Type,Description,Location Description,Arrest,Domestic,Beat,District,...,Year,Updated On,Location,Day,MinTemp,MaxTemp,Season,AvgTemp,Weather,Inside/Outside
0,10000092,2015-03-18 19:44:00,047XX W OHIO ST,BATTERY,AGGRAVATED: HANDGUN,STREET,False,False,1111,11.0,...,2015,02/10/2018 03:50:01 PM,"(41.891398861, -87.744384567)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Outside
1,10000094,2015-03-18 23:00:00,066XX S MARSHFIELD AVE,OTHER OFFENSE,PAROLE VIOLATION,STREET,True,False,725,7.0,...,2015,02/10/2018 03:50:01 PM,"(41.773371528, -87.665319468)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Outside
2,10000095,2015-03-18 22:45:00,044XX S LAKE PARK AVE,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,False,True,222,2.0,...,2015,02/10/2018 03:50:01 PM,"(41.81386068, -87.596642837)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Inside
3,10000096,2015-03-18 22:30:00,051XX S MICHIGAN AVE,BATTERY,SIMPLE,APARTMENT,False,False,225,2.0,...,2015,02/10/2018 03:50:01 PM,"(41.800802415, -87.622619343)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Inside
4,10000097,2015-03-18 21:00:00,047XX W ADAMS ST,ROBBERY,ARMED: HANDGUN,SIDEWALK,False,False,1113,11.0,...,2015,02/10/2018 03:50:01 PM,"(41.878064761, -87.743354013)",2015-03-18,34,48,Spring,41.0,30-50 degrees,Outside


In [33]:
df2.to_csv('Data/TotalData2', index = False)

In [34]:
temps = weather.loc[: , "MinTemp":"MaxTemp"]
weather['AvgTemp'] = temps.mean(axis=1)
temp = [-200,30,50,70,90,200]
temp_labels = ['Below freezing', '30-50 degrees', '50-70 degrees', '70-90 degrees', 'Over 90 degrees']
weather['Ranges'] = pd.cut(weather['AvgTemp'], temp, labels=temp_labels)
weather.head()

,Day,MinTemp,MaxTemp,AvgTemp,Ranges
0,2010-01-01,13,19,16.0,Below freezing
1,2010-01-02,9,13,11.0,Below freezing
2,2010-01-03,12,20,16.0,Below freezing
3,2010-01-04,15,23,19.0,Below freezing
4,2010-01-05,20,25,22.5,Below freezing


In [35]:
weather.groupby('Ranges')['Ranges'].count() / weather['Day'].nunique()

Ranges
Below freezing     0.146192
30-50 degrees      0.319717
50-70 degrees      0.290541
70-90 degrees      0.241093
Over 90 degrees    0.002457
Name: Ranges, dtype: float64

In [36]:
weather['Day'].nunique()

3256

In [37]:
# we have a total of 3256 dates involved
#Below freezing      476
#30-50 degrees      1041
#50-70 degrees       946
#70-90 degrees       785
#Over 90 degrees       8
# which gives us 
#Below freezing     0.146192
#30-50 degrees      0.319717
#50-70 degrees      0.290541
#70-90 degrees      0.241093
#Over 90 degrees    0.002457

In [38]:
(weather.groupby('Ranges')['Ranges'].count() / weather['Day'].nunique()) * 2683565

Ranges
Below freezing     392314.785012
30-50 degrees      857982.544533
50-70 degrees      779684.425676
70-90 degrees      646989.718980
Over 90 degrees      6593.525799
Name: Ranges, dtype: float64

In [39]:
weather.to_csv('Data/WeatherData', index = False)